In [1]:
import os
import pprint
import absl
import tensorflow as tf
import tensorflow_data_validation as tfdv
import tensorflow_model_analysis as tfma
import tfx
from tfx.components import CsvExampleGen
from tfx.components import StatisticsGen
from tfx.components import SchemaGen
from tfx.components import ExampleValidator
from tfx.components import Transform
from tfx.components import Tuner
from tfx.components import Trainer
from tfx.components import Evaluator
from tfx.components import Pusher
from tfx.v1.dsl import Resolver
from tfx.v1.dsl import Importer
from tfx.components.base import executor_spec
from tfx.components.trainer.executor import GenericExecutor
from tfx.dsl.experimental import latest_blessed_model_resolver
from tfx.orchestration import metadata
from tfx.orchestration import pipeline
from tfx.orchestration.experimental.interactive.interactive_context import InteractiveContext
from tfx.proto import pusher_pb2
from tfx.proto import trainer_pb2
from tfx.types import Channel
from tfx.types.standard_artifacts import HyperParameters
from tfx.types.standard_artifacts import Model
from tfx.types.standard_artifacts import ModelBlessing
from google.protobuf import text_format
from tensorflow_data_validation.utils import io_util
from tensorflow_metadata.proto.v0 import anomalies_pb2

tf.get_logger().propagate = False
pp = pprint.PrettyPrinter()

%load_ext tfx.orchestration.experimental.interactive.notebook_extensions.skip

In [2]:
# 호환되는 버전
# !pip install -U pip
# !pip install future
# !pip install -U tfx==1.3
# !pip install --upgrade tensorflow-estimator==2.6.0
# !pip install --upgrade keras==2.6.0

In [3]:
# TFMA 설정
# jupyter nbextension enable --py widgetsnbextension --sys-prefix 
# jupyter nbextension install --py --symlink tensorflow_model_analysis --sys-prefix 
# jupyter nbextension enable --py tensorflow_model_analysis --sys-prefix

In [4]:
_DATA_ROOT = './data'
_DATA_FILEPATH = os.path.join(_DATA_ROOT, "advertising.csv")

_PIPELINE_ROOT = './pipeline'
_PIPELINE_NAME = 'advert-pipeline'
_META_DATA_PATH = os.path.join(_PIPELINE_ROOT, "metadata.db")

In [5]:
print(tf.__version__)

2.6.0


## Helper functions

#### 참조
- Production 환경에서는 MLMD가 들어있는 Metadata store에서 정보들을 가져옴
- SQLite나 Mysql 객체에 연결해서 쿼리로 로그처럼 조회 가능
    - https://www.tensorflow.org/tfx/guide/mlmd?hl=ko
- Interactive context 환경에서 store 만들고 확인하는 케이스
    - 직접적인 artifact 내용 보는 것이 아닌 lineage 확인이나 연관된 요소 확인 위한 파이썬 코드 방식 예시
    - https://www.tensorflow.org/tfx/tutorials/mlmd/mlmd_tutorial

In [6]:
# ExampleGen, Transformation 결과 확인
def load_sample_tfrecord_from_uri(uri, sample_number, compression_type="GZIP"):
    tfrecord_filenames = [os.path.join(uri, name)
                         for name in os.listdir(uri)]
    dataset = tf.data.TFRecordDataset(tfrecord_filenames, compression_type)
    
    for tfrecord in dataset.take(sample_number):
        serialized_example = tfrecord.numpy()
        example = tf.train.Example()
        example.ParseFromString(serialized_example)
        print(example)

In [7]:
# StatisticsGen
def load_stats_from_uri(uri):
    file_name = os.listdir(uri)[0]
    file_format = file_name.split('.')[1]
    file_path = os.path.join(uri, file_name)
    
    if file_format == 'pb':
        result = tfdv.load_stats_binary(file_path)
    elif file_format == 'pbtxt':
        result = tfdv.load_stats_text(file_path)
    else:
        result = None
    
    return result


# SchemaGen
def load_schema_from_uri(uri):
    file_name = os.listdir(uri)[0]
    file_path = os.path.join(uri, file_name)
    
    result = tfdv.load_schema_text(file_path)
    
    return result


# ExampleValidator
def load_anomalies_from_uri(uri):
    result = None
    file_name = os.listdir(uri)[0]
    file_format = file_name.split('.')[1]
    file_path = os.path.join(uri, file_name)
    
    if file_format == 'pb':
        # Source code
        anomalies_proto = anomalies_pb2.Anomalies()
        anomalies_proto.ParseFromString(io_util.read_file_to_string(
          file_path, binary_mode=True))  # binary mode true로 해서 parsing
        result = anomalies_proto
        
    elif file_format == 'pbtxt':
        result = tfdv.load_anomalies_binary(file_path)
        # Source code
#         anomalies = anomalies_pb2.Anomalies()
#         anomalies_text = io_util.read_file_to_string(input_path)
#         text_format.Parse(anomalies_text, anomalies)
#         result = anomalies
        
    return result

## Interactive Context 환경 생성

In [8]:
# Interactive context에서는 로컬에 지정된 pipline_root로 artifact들이 생성됨
# Production pipeline 구성 시에는 google storage, S3 같은 object storage에 저장
context = InteractiveContext(pipeline_root=_PIPELINE_ROOT,
                             pipeline_name=_PIPELINE_NAME,
                             metadata_connection_config=tfx.orchestration.metadata.sqlite_metadata_connection_config(_META_DATA_PATH))

## TFDV

### ExampleGen

In [9]:
example_gen = CsvExampleGen(input_base=_DATA_ROOT)

In [10]:
context.run(example_gen)  # run 해야 생성됨

/Users/jinwoohong/.pyenv/versions/tfx/lib/python3.8/site-packages/setuptools/distutils_patch.py:25: UserWarning: Distutils was imported before Setuptools. This usage is discouraged and may exhibit undesirable behaviors or errors. Please use Setuptools' objects directly or at least import Setuptools first.
  warnings.warn(


ExecutionResult(
    component_id: CsvExampleGen
    execution_id: 1
    outputs:
        examples: Channel(
            type_name: Examples
            artifacts: [Artifact(artifact: id: 1
        type_id: 14
        uri: "./pipeline/CsvExampleGen/examples/1"
        properties {
          key: "split_names"
          value {
            string_value: "[\"train\", \"eval\"]"
          }
        }
        custom_properties {
          key: "file_format"
          value {
            string_value: "tfrecords_gzip"
          }
        }
        custom_properties {
          key: "input_fingerprint"
          value {
            string_value: "split:single_split,num_files:1,total_bytes:107413,xor_checksum:1649997737,sum_checksum:1649997737"
          }
        }
        custom_properties {
          key: "payload_format"
          value {
            string_value: "FORMAT_TF_EXAMPLE"
          }
        }
        custom_properties {
          key: "span"
          value {
            int_value: 0
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "1.3.0"
          }
        }
        state: LIVE
        , artifact_type: id: 14
        name: "Examples"
        properties {
          key: "span"
          value: INT
        }
        properties {
          key: "split_names"
          value: STRING
        }
        properties {
          key: "version"
          value: INT
        }
        )]
            additional_properties: {}
            additional_custom_properties: {}
        ))

In [11]:
# artifcat 자체 접근
artifact = example_gen.outputs['examples'].get()[0]  

In [12]:
print(artifact.split_names, artifact.uri)

["train", "eval"] ./pipeline/CsvExampleGen/examples/1


In [13]:
artifact

.type,<class 'tfx.types.standard_artifacts.Examples'>
.uri,./pipeline/CsvExampleGen/examples/1
.span,0
.split_names,"[""train"", ""eval""]"
.version,0


In [14]:
# 샘플 예시 보기
# Get the URI of the output artifact representing the transformed examples
raw_train_uri = os.path.join(example_gen.outputs['examples'].get()[0].uri, 'Split-train')

load_sample_tfrecord_from_uri(raw_train_uri, 1)

2022-04-22 15:05:02.678467: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


features {
  feature {
    key: "AdTopicLine"
    value {
      bytes_list {
        value: "Cloned 5thgeneration orchestration"
      }
    }
  }
  feature {
    key: "Age"
    value {
      int64_list {
        value: 35
      }
    }
  }
  feature {
    key: "AreaIncome"
    value {
      float_list {
        value: 61833.8984375
      }
    }
  }
  feature {
    key: "City"
    value {
      bytes_list {
        value: "Wrightburgh"
      }
    }
  }
  feature {
    key: "ClickedOnAd"
    value {
      int64_list {
        value: 0
      }
    }
  }
  feature {
    key: "Country"
    value {
      bytes_list {
        value: "Tunisia"
      }
    }
  }
  feature {
    key: "DailyInternetUsage"
    value {
      float_list {
        value: 256.0899963378906
      }
    }
  }
  feature {
    key: "DailyTimeSpentOnSite"
    value {
      float_list {
        value: 68.94999694824219
      }
    }
  }
  feature {
    key: "Male"
    value {
      int64_list {
        value: 0
      }
 

2022-04-22 15:05:02.700634: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


### StatisticsGen

In [15]:
statistics_gen = StatisticsGen(
   examples=example_gen.outputs['examples'])

In [16]:
context.run(statistics_gen)

ExecutionResult(
    component_id: StatisticsGen
    execution_id: 2
    outputs:
        statistics: Channel(
            type_name: ExampleStatistics
            artifacts: [Artifact(artifact: id: 2
        type_id: 16
        uri: "./pipeline/StatisticsGen/statistics/2"
        properties {
          key: "split_names"
          value {
            string_value: "[\"train\", \"eval\"]"
          }
        }
        custom_properties {
          key: "name"
          value {
            string_value: "statistics"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "StatisticsGen"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "1.3.0"
          }
        }
        state: LIVE
        , artifact_type: id: 16
        name: "ExampleStatistics"
        properties {
          key: "span"
          value: INT
        }
        properties {
          key: "split_names"
          value: STRING
        }
        )]
            additional_properties: {}
            additional_custom_properties: {}
        ))

In [17]:
# 아래 코드와 같음
# statistics_gen.outputs['statistics']._artifacts
# statistics_gen.outputs['statistics'].get()
stats_uri = statistics_gen.outputs['statistics'].get()[0].uri
stats_uri

'./pipeline/StatisticsGen/statistics/2'

In [18]:
# 값 접근 가능
# load_stats_from_uri(os.path.join(stats_uri, 'Split-eval'))

In [19]:
statistics_gen.outputs['statistics']

Channel(
    type_name: ExampleStatistics
    artifacts: [Artifact(artifact: id: 2
type_id: 16
uri: "./pipeline/StatisticsGen/statistics/2"
properties {
  key: "split_names"
  value {
    string_value: "[\"train\", \"eval\"]"
  }
}
custom_properties {
  key: "name"
  value {
    string_value: "statistics"
  }
}
custom_properties {
  key: "producer_component"
  value {
    string_value: "StatisticsGen"
  }
}
custom_properties {
  key: "state"
  value {
    string_value: "published"
  }
}
custom_properties {
  key: "tfx_version"
  value {
    string_value: "1.3.0"
  }
}
state: LIVE
, artifact_type: id: 16
name: "ExampleStatistics"
properties {
  key: "span"
  value: INT
}
properties {
  key: "split_names"
  value: STRING
}
)]
    additional_properties: {}
    additional_custom_properties: {}
)

In [20]:
context.show(statistics_gen.outputs['statistics'])

### SchemaGen

In [21]:
schema_gen = SchemaGen(
   statistics=statistics_gen.outputs['statistics'],
   infer_feature_shape=False)  # False로 해야 나중에 missing 값 나올 때 조정해도 shape anomalies 안 나옴. 꼭 지정해야 하는 shape는 나중에 따로 지정

In [22]:
context.run(schema_gen)

I0422 15:05:04.714113 420371904 rdbms_metadata_access_object.cc:686] No property is defined for the Type


ExecutionResult(
    component_id: SchemaGen
    execution_id: 3
    outputs:
        schema: Channel(
            type_name: Schema
            artifacts: [Artifact(artifact: id: 3
        type_id: 18
        uri: "./pipeline/SchemaGen/schema/3"
        custom_properties {
          key: "name"
          value {
            string_value: "schema"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "SchemaGen"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "1.3.0"
          }
        }
        state: LIVE
        , artifact_type: id: 18
        name: "Schema"
        )]
            additional_properties: {}
            additional_custom_properties: {}
        ))

In [23]:
schema_uri = schema_gen.outputs['schema'].get()[0].uri
schema_uri

'./pipeline/SchemaGen/schema/3'

In [24]:
# 값 접근 가능
# load_schema_from_uri(schema_uri)

In [25]:
context.show(schema_gen.outputs['schema'])

,Type,Presence,Valency,Domain
Feature name,,,,
'AdTopicLine',BYTES,required,single,-
'City',BYTES,required,single,-
'Country',BYTES,required,single,-
'Timestamp',BYTES,required,single,-
'Age',INT,required,single,-
'AreaIncome',FLOAT,required,single,-
'ClickedOnAd',INT,required,single,-
'DailyInternetUsage',FLOAT,required,single,-
'DailyTimeSpentOnSite',FLOAT,required,single,-


### ExampleValidator

In [26]:
example_validator = ExampleValidator(
   statistics=statistics_gen.outputs['statistics'],
   schema=schema_gen.outputs['schema'])

In [27]:
example_validator

ExampleValidator(spec: <tfx.types.standard_component_specs.ExampleValidatorSpec object at 0x1a64ab4c0>, executor_spec: <tfx.dsl.components.base.executor_spec.ExecutorClassSpec object at 0x1a64ab520>, driver_class: <class 'tfx.dsl.components.base.base_driver.BaseDriver'>, component_id: ExampleValidator, inputs: {'statistics': Channel(
    type_name: ExampleStatistics
    artifacts: [Artifact(artifact: id: 2
type_id: 16
uri: "./pipeline/StatisticsGen/statistics/2"
properties {
  key: "split_names"
  value {
    string_value: "[\"train\", \"eval\"]"
  }
}
custom_properties {
  key: "name"
  value {
    string_value: "statistics"
  }
}
custom_properties {
  key: "producer_component"
  value {
    string_value: "StatisticsGen"
  }
}
custom_properties {
  key: "state"
  value {
    string_value: "published"
  }
}
custom_properties {
  key: "tfx_version"
  value {
    string_value: "1.3.0"
  }
}
state: LIVE
, artifact_type: id: 16
name: "ExampleStatistics"
properties {
  key: "span"
  value: INT
}
properties {
  key: "split_names"
  value: STRING
}
)]
    additional_properties: {}
    additional_custom_properties: {}
), 'schema': Channel(
    type_name: Schema
    artifacts: [Artifact(artifact: id: 3
type_id: 18
uri: "./pipeline/SchemaGen/schema/3"
custom_properties {
  key: "name"
  value {
    string_value: "schema"
  }
}
custom_properties {
  key: "producer_component"
  value {
    string_value: "SchemaGen"
  }
}
custom_properties {
  key: "state"
  value {
    string_value: "published"
  }
}
custom_properties {
  key: "tfx_version"
  value {
    string_value: "1.3.0"
  }
}
state: LIVE
, artifact_type: id: 18
name: "Schema"
)]
    additional_properties: {}
    additional_custom_properties: {}
)}, outputs: {'anomalies': Channel(
    type_name: ExampleAnomalies
    artifacts: []
    additional_properties: {}
    additional_custom_properties: {}
)})

In [28]:
context.run(example_validator)

ExecutionResult(
    component_id: ExampleValidator
    execution_id: 4
    outputs:
        anomalies: Channel(
            type_name: ExampleAnomalies
            artifacts: [Artifact(artifact: id: 4
        type_id: 20
        uri: "./pipeline/ExampleValidator/anomalies/4"
        properties {
          key: "split_names"
          value {
            string_value: "[\"train\", \"eval\"]"
          }
        }
        custom_properties {
          key: "name"
          value {
            string_value: "anomalies"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "ExampleValidator"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "1.3.0"
          }
        }
        state: LIVE
        , artifact_type: id: 20
        name: "ExampleAnomalies"
        properties {
          key: "span"
          value: INT
        }
        properties {
          key: "split_names"
          value: STRING
        }
        )]
            additional_properties: {}
            additional_custom_properties: {}
        ))

In [29]:
example_validator_uri = example_validator.outputs['anomalies'].get()[0].uri
example_validator_uri

'./pipeline/ExampleValidator/anomalies/4'

In [30]:
# 값 확인 가능
# load_anomalies_from_uri(os.path.join(example_validator_uri, 'Split-eval'))

In [31]:
context.show(example_validator.outputs['anomalies'])

/Users/jinwoohong/.pyenv/versions/tfx/lib/python3.8/site-packages/tensorflow_data_validation/utils/display_util.py:217: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option('max_colwidth', -1)


/Users/jinwoohong/.pyenv/versions/tfx/lib/python3.8/site-packages/tensorflow_data_validation/utils/display_util.py:217: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option('max_colwidth', -1)


In [32]:
example_validator.outputs['anomalies']

Channel(
    type_name: ExampleAnomalies
    artifacts: [Artifact(artifact: id: 4
type_id: 20
uri: "./pipeline/ExampleValidator/anomalies/4"
properties {
  key: "split_names"
  value {
    string_value: "[\"train\", \"eval\"]"
  }
}
custom_properties {
  key: "name"
  value {
    string_value: "anomalies"
  }
}
custom_properties {
  key: "producer_component"
  value {
    string_value: "ExampleValidator"
  }
}
custom_properties {
  key: "state"
  value {
    string_value: "published"
  }
}
custom_properties {
  key: "tfx_version"
  value {
    string_value: "1.3.0"
  }
}
state: LIVE
, artifact_type: id: 20
name: "ExampleAnomalies"
properties {
  key: "span"
  value: INT
}
properties {
  key: "split_names"
  value: STRING
}
)]
    additional_properties: {}
    additional_custom_properties: {}
)

## Transform

In [33]:
advert_transform = 'model/advert_transform.py'
transform = Transform(
    examples=example_gen.outputs['examples'],
    schema=schema_gen.outputs['schema'],
    module_file=advert_transform)

In [34]:
context.run(transform)

running bdist_wheel
running build
running build_py
creating build
creating build/lib
copying advert_transform.py -> build/lib
copying advert_trainer_backup.py -> build/lib
copying advert_trainer.py -> build/lib
copying advert_tuner.py -> build/lib
copying advert_constants.py -> build/lib
installing to /var/folders/4b/sf_49brs005d5sy8ql2r7yq80000gn/T/tmpx73ev3er
running install
running install_lib
copying build/lib/advert_transform.py -> /var/folders/4b/sf_49brs005d5sy8ql2r7yq80000gn/T/tmpx73ev3er
copying build/lib/advert_trainer_backup.py -> /var/folders/4b/sf_49brs005d5sy8ql2r7yq80000gn/T/tmpx73ev3er
copying build/lib/advert_trainer.py -> /var/folders/4b/sf_49brs005d5sy8ql2r7yq80000gn/T/tmpx73ev3er
copying build/lib/advert_tuner.py -> /var/folders/4b/sf_49brs005d5sy8ql2r7yq80000gn/T/tmpx73ev3er
copying build/lib/advert_constants.py -> /var/folders/4b/sf_49brs005d5sy8ql2r7yq80000gn/T/tmpx73ev3er
running install_egg_info
running egg_info
creating tfx_user_code_Transform.egg-info
writing

I0422 15:05:06.474767 420371904 rdbms_metadata_access_object.cc:686] No property is defined for the Type
I0422 15:05:06.475878 420371904 rdbms_metadata_access_object.cc:686] No property is defined for the Type


Processing ./pipeline/_wheels/tfx_user_code_Transform-0.0+a141aba1922445b5623f64bd134d2268aeecedab6f142166fb94958ee4a96629-py3-none-any.whl
Processing ./pipeline/_wheels/tfx_user_code_Transform-0.0+a141aba1922445b5623f64bd134d2268aeecedab6f142166fb94958ee4a96629-py3-none-any.whl
Instructions for updating:
Use ref() instead.
Processing ./pipeline/_wheels/tfx_user_code_Transform-0.0+a141aba1922445b5623f64bd134d2268aeecedab6f142166fb94958ee4a96629-py3-none-any.whl


2022-04-22 15:05:16.398007: W tensorflow/python/util/util.cc:348] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


INFO:tensorflow:Assets written to: ./pipeline/Transform/transform_graph/5/.temp_path/tftransform_tmp/f8ed739ab73e4cdcb4ab227c3d84c4fa/assets
INFO:tensorflow:tensorflow_text is not available.
INFO:tensorflow:tensorflow_decision_forests is not available.
INFO:tensorflow:struct2tensor is not available.
INFO:tensorflow:Assets written to: ./pipeline/Transform/transform_graph/5/.temp_path/tftransform_tmp/fc78030e4e1d4423827c8087de6bc15e/assets
INFO:tensorflow:tensorflow_text is not available.
INFO:tensorflow:tensorflow_decision_forests is not available.
INFO:tensorflow:struct2tensor is not available.
INFO:tensorflow:tensorflow_text is not available.
INFO:tensorflow:tensorflow_decision_forests is not available.
INFO:tensorflow:struct2tensor is not available.


ExecutionResult(
    component_id: Transform
    execution_id: 5
    outputs:
        transform_graph: Channel(
            type_name: TransformGraph
            artifacts: [Artifact(artifact: id: 5
        type_id: 22
        uri: "./pipeline/Transform/transform_graph/5"
        custom_properties {
          key: "name"
          value {
            string_value: "transform_graph"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "Transform"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "1.3.0"
          }
        }
        state: LIVE
        , artifact_type: id: 22
        name: "TransformGraph"
        )]
            additional_properties: {}
            additional_custom_properties: {}
        )
        transformed_examples: Channel(
            type_name: Examples
            artifacts: [Artifact(artifact: id: 6
        type_id: 14
        uri: "./pipeline/Transform/transformed_examples/5"
        properties {
          key: "split_names"
          value {
            string_value: "[\"train\", \"eval\"]"
          }
        }
        custom_properties {
          key: "name"
          value {
            string_value: "transformed_examples"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "Transform"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "1.3.0"
          }
        }
        state: LIVE
        , artifact_type: id: 14
        name: "Examples"
        properties {
          key: "span"
          value: INT
        }
        properties {
          key: "split_names"
          value: STRING
        }
        properties {
          key: "version"
          value: INT
        }
        )]
            additional_properties: {}
            additional_custom_properties: {}
        )
        updated_analyzer_cache: Channel(
            type_name: TransformCache
            artifacts: [Artifact(artifact: id: 7
        type_id: 23
        uri: "./pipeline/Transform/updated_analyzer_cache/5"
        custom_properties {
          key: "name"
          value {
            string_value: "updated_analyzer_cache"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "Transform"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "1.3.0"
          }
        }
        state: LIVE
        , artifact_type: id: 23
        name: "TransformCache"
        )]
            additional_properties: {}
            additional_custom_properties: {}
        )
        pre_transform_schema: Channel(
            type_name: Schema
            artifacts: [Artifact(artifact: id: 8
        type_id: 18
        uri: "./pipeline/Transform/pre_transform_schema/5"
        custom_properties {
          key: "name"
          value {
            string_value: "pre_transform_schema"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "Transform"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "1.3.0"
          }
        }
        state: LIVE
        , a

In [35]:
transform.outputs['transformed_examples'].get()[0]

.type,<class 'tfx.types.standard_artifacts.Examples'>
.uri,./pipeline/Transform/transformed_examples/5
.span,0
.split_names,"[""train"", ""eval""]"
.version,0


In [36]:
# 샘플 예시 보기
# Get the URI of the output artifact representing the transformed examples
train_uri = os.path.join(transform.outputs['transformed_examples'].get()[0].uri, 'Split-train')

# Get the list of files in this directory (all compressed TFRecord files)
tfrecord_filenames = [os.path.join(train_uri, name)
                      for name in os.listdir(train_uri)]

# Create a `TFRecordDataset` to read these files
dataset = tf.data.TFRecordDataset(tfrecord_filenames, compression_type="GZIP")

# Decode the first record and print output
for tfrecord in dataset.take(1):
    serialized_example = tfrecord.numpy()
    example = tf.train.Example()
    example.ParseFromString(serialized_example)
#     pp.pprint(example)

## Tuner

Step
- Tuner를 통해 최적의 neural architecture을 검색해 산출
- 산출된 best hyper parameter를 통해 training

TODO: 일단 나중에 다시 적용해보자

In [37]:
_tuner_module_file = './model/advert_tuner.py'

In [38]:
%%writefile {_tuner_module_file}
from kerastuner.engine import base_tuner
import kerastuner as kt
from tensorflow import keras
from typing import NamedTuple, Dict, Text, Any, List
from tfx.components.trainer.fn_args_utils import FnArgs, DataAccessor
import tensorflow as tf
import tensorflow_transform as tft
import advert_constants as constants

# Declare namedtuple field names
TunerFnResult = NamedTuple('TunerFnResult', [('tuner', base_tuner.BaseTuner),
                                             ('fit_kwargs', Dict[Text, Any])])
# Callback for the search strategy
stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)

_DENSE_FLOAT_FEATURE_KEYS = constants.DENSE_FLOAT_FEATURE_KEYS
_VOCAB_FEATURE_KEYS = constants.VOCAB_FEATURE_KEYS
_VOCAB_SIZE = constants.VOCAB_SIZE
_OOV_SIZE = constants.OOV_SIZE
_LABEL_KEY = constants.LABEL_KEY
_transformed_name = constants.transformed_name


def _transformed_names(keys):
    return [_transformed_name(key) for key in keys]


def _gzip_reader_fn(filenames):
    """Load compressed dataset

    Args:
      filenames - filenames of TFRecords to load

    Returns:
      TFRecordDataset loaded from the filenames
    """

    # Load the dataset. Specify the compression type since it is saved as `.gz`
    return tf.data.TFRecordDataset(filenames, compression_type='GZIP')


def _input_fn(file_pattern,
              tf_transform_output,
              num_epochs=None,
              batch_size=32) -> tf.data.Dataset:
    """Create batches of features and labels from TF Records

    Args:
      file_pattern - List of files or patterns of file paths containing Example records.
      tf_transform_output - transform output graph
      num_epochs - Integer specifying the number of times to read through the dataset.
              If None, cycles through the dataset forever.
      batch_size - An int representing the number of records to combine in a single batch.

    Returns:
      A dataset of dict elements, (or a tuple of dict elements and label).
      Each dict maps feature keys to Tensor or SparseTensor objects.
    """
    transformed_feature_spec = (
        tf_transform_output.transformed_feature_spec().copy())

    dataset = tf.data.experimental.make_batched_features_dataset(
        file_pattern=file_pattern,
        batch_size=batch_size,
        features=transformed_feature_spec,
        reader=_gzip_reader_fn,
        num_epochs=num_epochs,
        label_key=_transformed_name(_LABEL_KEY))

    return dataset


def _model_builder(hp):
    """
    Builds the model and sets up the hyperparameters to tune.

    Args:
      hp - Keras tuner object

    Returns:
      model with hyperparameters to tune
    """
    
    num_dnn_layers = 3  # 일단 복잡하니까 작은 수로 지정하고 optimize도 제외
    hp_first_units = hp.Int('units', min_value=128, max_value=256, step=32)
    hp_decay_factor = hp.Choice('decay', values=[0.7, 0.5, 0.3])
    
    hidden_units = [
         max(2, int(hp_first_units * hp_decay_factor**i))
         for i in range(num_dnn_layers)
    ]
    
    model = _wide_and_deep_classifier(hidden_units)
    
    return model


def _wide_and_deep_classifier(hidden_units):    
    input_numeric = [
        tf.keras.layers.Input(name=colname, shape=(1,), dtype=tf.float32)
        for colname in _transformed_names(_DENSE_FLOAT_FEATURE_KEYS)
    ]
    input_categorical = [
        tf.keras.layers.Input(name=colname, shape=(_VOCAB_SIZE + _OOV_SIZE,), dtype=tf.float32)
        for colname in _transformed_names(_VOCAB_FEATURE_KEYS)
    ]
    
    # TODO: Tuner hyperparameters로 받아오게 변경
    # deep 부분 정의
    deep = tf.keras.layers.concatenate(input_numeric)
    for numnodes in hidden_units:
        deep = tf.keras.layers.Dense(numnodes, name='deep_'+str(numnodes))(deep)
    # wide 부분 정의
    wide = tf.keras.layers.concatenate(input_categorical)
    # concat
    combined = keras.layers.concatenate([wide, deep], name='combined')
    # output 만들기
    output = tf.keras.layers.Dense(1, activation='sigmoid', name='output')(combined)
    # input layers와 output을 넣어 model 만들기
    # 이런 식으로 만들면 input layer에서 분리되서 wide, deep 나뉘고 다시 output으로 합쳐지게 깔끔하게 나옴
    input_layers = input_numeric + input_categorical
    model = tf.keras.Model(input_layers, output)
    
    # TODO: learning rate Tuner hyperparameters로 변경
    # Setup model for training
    model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.01),
                  loss="binary_crossentropy",
                  metrics=[tf.keras.metrics.BinaryAccuracy()])

    # Print the model summary
    model.summary()
    
    return model


def tuner_fn(fn_args: FnArgs) -> None:
    # Define tuner search strategy
    tuner = kt.Hyperband(_model_builder,
                     objective='binary_accuracy',
                     max_epochs=10,
                     factor=3,
                     directory=fn_args.working_dir,
                     project_name='kt_hyperband')
    
    # Load transform output
    tf_transform_output = tft.TFTransformOutput(fn_args.transform_graph_path)
    
    # epochs 설정할 수 있는데 steps 쓰면 steps이 epochs*원래스탭 보다 클 수도 있으므로 None인 기본값으로 해 계속 순회하도록 함
    # train_steps, eval_steps을 안 쓰고 epochs으로 조정하는 방법도 있음
    train_set = _input_fn(fn_args.train_files[0], tf_transform_output)  
    val_set = _input_fn(fn_args.eval_files[0], tf_transform_output)
    
    return TunerFnResult(
    tuner=tuner,
    fit_kwargs={ 
      "callbacks":[stop_early],
      'x': train_set,
      'validation_data': val_set,
      'steps_per_epoch': fn_args.train_steps,
      'validation_steps': fn_args.eval_steps}
    )   

Overwriting ./model/advert_tuner.py


In [39]:
# Setup the Tuner component
tuner = Tuner(
    module_file=_tuner_module_file,
    examples=transform.outputs['transformed_examples'],
    transform_graph=transform.outputs['transform_graph'],
    schema=schema_gen.outputs['schema'],
    train_args=trainer_pb2.TrainArgs(splits=['train'], num_steps=500),
    eval_args=trainer_pb2.EvalArgs(splits=['eval'], num_steps=100)
    )

In [40]:
context.run(tuner, enable_cache=False)

Trial 21 Complete [00h 00m 03s]
binary_accuracy: 1.0

Best binary_accuracy So Far: 1.0
Total elapsed time: 00h 00m 55s
INFO:tensorflow:Oracle triggered exit
Results summary
Results in ./pipeline/.temp/6/kt_hyperband
Showing 10 best trials
Trial summary
Hyperparameters:
units: 224
decay: 0.3
tuner/epochs: 10
tuner/initial_epoch: 4
tuner/bracket: 2
tuner/round: 2
tuner/trial_id: 0014
Score: 1.0
Trial summary
Hyperparameters:
units: 128
decay: 0.3
tuner/epochs: 10
tuner/initial_epoch: 4
tuner/bracket: 2
tuner/round: 2
tuner/trial_id: 0012
Score: 1.0
Trial summary
Hyperparameters:
units: 224
decay: 0.5
tuner/epochs: 4
tuner/initial_epoch: 0
tuner/bracket: 1
tuner/round: 0
Score: 1.0
Trial summary
Hyperparameters:
units: 224
decay: 0.7
tuner/epochs: 4
tuner/initial_epoch: 0
tuner/bracket: 1
tuner/round: 0
Score: 1.0
Trial summary
Hyperparameters:
units: 192
decay: 0.3
tuner/epochs: 4
tuner/initial_epoch: 0
tuner/bracket: 1
tuner/round: 0
Score: 0.999875009059906
Trial summary
Hyperparameter

ExecutionResult(
    component_id: Tuner
    execution_id: 6
    outputs:
        best_hyperparameters: Channel(
            type_name: HyperParameters
            artifacts: [Artifact(artifact: id: 13
        type_id: 25
        uri: "./pipeline/Tuner/best_hyperparameters/6"
        custom_properties {
          key: "name"
          value {
            string_value: "best_hyperparameters"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "Tuner"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "1.3.0"
          }
        }
        state: LIVE
        , artifact_type: id: 25
        name: "HyperParameters"
        )]
            additional_properties: {}
            additional_custom_properties: {}
        ))

In [41]:
tuner_output_dir = tuner.outputs['best_hyperparameters'].get()[0].uri

In [42]:
os.path.join(tuner_output_dir, 'best_hyperparameters.txt')

'./pipeline/Tuner/best_hyperparameters/6/best_hyperparameters.txt'

In [43]:
# TODO: 잘 작동하는지 확인
# 버전 업하면서 depreciated된 것 같음
hparams_importer = Importer(
    # This can be Tuner's output file or manually edited file. The file contains
    # text format of hyperparameters (kerastuner.HyperParameters.get_config())
    source_uri=os.path.join(tuner_output_dir, 'best_hyperparameters.txt'),
    artifact_type=HyperParameters)

In [44]:
hparams_importer.outputs['result'].get()[0]

.type,<class 'tfx.types.standard_artifacts.HyperParameters'>
.uri,


## Trainer

Test 시에는 아래처럼 쥬피터 상에서 overwrite을 해야 쥬피터 상에서 반영됨. 다른 환경에서 py 파일 고쳐도 적용 안 될 때 있음

### Tuner 활용한 training

In [45]:
_trainer_module_file = './model/advert_trainer.py'

In [46]:
%%writefile {_trainer_module_file}
from tensorflow import keras
from typing import NamedTuple, Dict, Text, Any, List
from tfx.components.trainer.fn_args_utils import FnArgs, DataAccessor
import tensorflow as tf
import tensorflow_transform as tft
import advert_constants as constants


_DENSE_FLOAT_FEATURE_KEYS = constants.DENSE_FLOAT_FEATURE_KEYS
_VOCAB_FEATURE_KEYS = constants.VOCAB_FEATURE_KEYS
_VOCAB_SIZE = constants.VOCAB_SIZE
_OOV_SIZE = constants.OOV_SIZE
_LABEL_KEY = constants.LABEL_KEY
_transformed_name = constants.transformed_name


def _transformed_names(keys):
    return [_transformed_name(key) for key in keys]



def _gzip_reader_fn(filenames):
    """Load compressed dataset

    Args:
      filenames - filenames of TFRecords to load

    Returns:
      TFRecordDataset loaded from the filenames
    """

    # Load the dataset. Specify the compression type since it is saved as `.gz`
    return tf.data.TFRecordDataset(filenames, compression_type='GZIP')


def _input_fn(file_pattern,
              tf_transform_output,
              num_epochs=None,
              batch_size=32) -> tf.data.Dataset:
    """Create batches of features and labels from TF Records

    Args:
      file_pattern - List of files or patterns of file paths containing Example records.
      tf_transform_output - transform output graph
      num_epochs - Integer specifying the number of times to read through the dataset.
              If None, cycles through the dataset forever.
      batch_size - An int representing the number of records to combine in a single batch.

    Returns:
      A dataset of dict elements, (or a tuple of dict elements and label).
      Each dict maps feature keys to Tensor or SparseTensor objects.
    """
    transformed_feature_spec = (
        tf_transform_output.transformed_feature_spec().copy())

    dataset = tf.data.experimental.make_batched_features_dataset(
        file_pattern=file_pattern,
        batch_size=batch_size,
        features=transformed_feature_spec,
        reader=_gzip_reader_fn,
        num_epochs=num_epochs,
        label_key=_transformed_name(_LABEL_KEY))

    return dataset


def _get_serve_tf_examples_fn(model, tf_transform_output):
    """Returns a function that parses a serialized tf.Example and applies TFT."""

    # Get transformation graph
    model.tft_layer = tf_transform_output.transform_features_layer()

    @tf.function
    def serve_tf_examples_fn(serialized_tf_examples):
        """Returns the output to be used in the serving signature."""
        # Get pre-transform feature spec
        feature_spec = tf_transform_output.raw_feature_spec()

        # Pop label since serving inputs do not include the label
        feature_spec.pop(_LABEL_KEY)

        # Parse raw examples into a dictionary of tensors matching the feature spec
        parsed_features = tf.io.parse_example(serialized_tf_examples, feature_spec)

        # Transform the raw examples using the transform graph
        transformed_features = model.tft_layer(parsed_features)

        # Get predictions using the transformed features
        return model(transformed_features)

    return serve_tf_examples_fn


def _model_builder(hp):
    """
    Builds the model and sets up the hyperparameters to tune.

    Args:
      hp - Keras tuner object

    Returns:
      model with hyperparameters to tune
    """
    num_dnn_layers = 3  # 일단 복잡하니까 작은 수로 지정하고 optimize도 제외
    hp_first_units = hp.get('units')
    hp_decay_factor = hp.get('decay')
    
    hidden_units = [
         max(2, int(hp_first_units * hp_decay_factor**i))
         for i in range(num_dnn_layers)
    ]
    
    model = _wide_and_deep_classifier(hidden_units)
    
    
    return model


def _wide_and_deep_classifier(hidden_units):    
    input_numeric = [
        tf.keras.layers.Input(name=colname, shape=(1,), dtype=tf.float32)
        for colname in _transformed_names(_DENSE_FLOAT_FEATURE_KEYS)
    ]
    input_categorical = [
        tf.keras.layers.Input(name=colname, shape=(_VOCAB_SIZE + _OOV_SIZE,), dtype=tf.float32)
        for colname in _transformed_names(_VOCAB_FEATURE_KEYS)
    ]
    
    # TODO: Tuner hyperparameters로 받아오게 변경
    # deep 부분 정의
    deep = tf.keras.layers.concatenate(input_numeric)
    for numnodes in hidden_units:
        deep = tf.keras.layers.Dense(numnodes, name='deep_'+str(numnodes))(deep)
    # wide 부분 정의
    wide = tf.keras.layers.concatenate(input_categorical)
    # concat
    combined = keras.layers.concatenate([wide, deep], name='combined')
    # output 만들기
    output = tf.keras.layers.Dense(1, activation='sigmoid', name='output')(combined)
    # input layers와 output을 넣어 model 만들기
    # 이런 식으로 만들면 input layer에서 분리되서 wide, deep 나뉘고 다시 output으로 합쳐지게 깔끔하게 나옴
    input_layers = input_numeric + input_categorical
    model = tf.keras.Model(input_layers, output)
    
    # TODO: learning rate Tuner hyperparameters로 변경
    # Setup model for training
    model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.01),
                  loss="binary_crossentropy",
                  metrics=[tf.keras.metrics.BinaryAccuracy()])

    # Print the model summary
    model.summary()
    
    return model


def run_fn(fn_args: FnArgs) -> None:
    """Defines and trains the model.
    Args:
      fn_args: Holds args as name/value pairs. Refer here for the complete attributes:
      https://www.tensorflow.org/tfx/api_docs/python/tfx/components/trainer/fn_args_utils/FnArgs#attributes
    """

    # Callback for TensorBoard
    tensorboard_callback = tf.keras.callbacks.TensorBoard(
        log_dir=fn_args.model_run_dir, update_freq='batch')

    # Load transform output
    tf_transform_output = tft.TFTransformOutput(fn_args.transform_graph_path)

    train_set = _input_fn(fn_args.train_files[0], tf_transform_output)
    val_set = _input_fn(fn_args.eval_files[0], tf_transform_output)
    
    # TODO: 여기서 tuner 이용하게 바꾸기
    # Load best hyperparameters
    # hp = fn_args.hyperparameters.get('values')
    # Load best hyperparameters
    hp = fn_args.hyperparameters.get('values')

    # Build the model
    model = _model_builder(hp)

    # Train the model
    model.fit(
        train_set,
        steps_per_epoch=fn_args.train_steps,  # 이렇게 해야 num_steps을 받아옴
        validation_data=val_set,
        validation_steps=fn_args.eval_steps,  # 이렇게 해야 num_steps을 받아옴
        callbacks=[tensorboard_callback])
    
    # Define default serving signature
    # TODO: 조금 더 제대로 이해해보기
    # SavedModel에 있는 하나의 특성으로 이를 바탕으로 raw data부터 전처리까지 한 후 추론이 가능하도록 만들 수 있음
    # _get_serve_tf_examples_fn 내에 정의된 serve_tf_example_fn(tf.function)으로 get_concrete_function을 통해 examples 데이터를 전달해 추론 가능하도록 함
    signatures = {
      'serving_default':
          _get_serve_tf_examples_fn(model,
                                    tf_transform_output).get_concrete_function(
                                        tf.TensorSpec(
                                            shape=[None],
                                            dtype=tf.string,
                                            name='examples')),
    }

    # Save the model
    model.save(fn_args.serving_model_dir, save_format='tf', signatures=signatures)

Overwriting ./model/advert_trainer.py


In [47]:
# Setup the Trainer component
trainer = Trainer(
    module_file=_trainer_module_file,
    examples=transform.outputs['transformed_examples'],
    hyperparameters=tuner.outputs['best_hyperparameters'],
    transform_graph=transform.outputs['transform_graph'],
    schema=schema_gen.outputs['schema'],
    train_args=trainer_pb2.TrainArgs(splits=['train'], num_steps=30000),
    eval_args=trainer_pb2.EvalArgs(splits=['eval'], num_steps=15000))

In [48]:
context.run(trainer)

I0422 15:06:20.709323 420371904 rdbms_metadata_access_object.cc:686] No property is defined for the Type
I0422 15:06:20.710212 420371904 rdbms_metadata_access_object.cc:686] No property is defined for the Type


running bdist_wheel
running build
running build_py
creating build
creating build/lib
copying advert_transform.py -> build/lib
copying advert_trainer_backup.py -> build/lib
copying advert_trainer.py -> build/lib
copying advert_tuner.py -> build/lib
copying advert_constants.py -> build/lib
installing to /var/folders/4b/sf_49brs005d5sy8ql2r7yq80000gn/T/tmp776g67e2
running install
running install_lib
copying build/lib/advert_transform.py -> /var/folders/4b/sf_49brs005d5sy8ql2r7yq80000gn/T/tmp776g67e2
copying build/lib/advert_trainer_backup.py -> /var/folders/4b/sf_49brs005d5sy8ql2r7yq80000gn/T/tmp776g67e2
copying build/lib/advert_trainer.py -> /var/folders/4b/sf_49brs005d5sy8ql2r7yq80000gn/T/tmp776g67e2
copying build/lib/advert_tuner.py -> /var/folders/4b/sf_49brs005d5sy8ql2r7yq80000gn/T/tmp776g67e2
copying build/lib/advert_constants.py -> /var/folders/4b/sf_49brs005d5sy8ql2r7yq80000gn/T/tmp776g67e2
running install_egg_info
running egg_info
creating tfx_user_code_Trainer.egg-info
writing t

2022-04-22 15:06:21.980355: I tensorflow/core/profiler/lib/profiler_session.cc:131] Profiler session initializing.
2022-04-22 15:06:21.980374: I tensorflow/core/profiler/lib/profiler_session.cc:146] Profiler session started.
2022-04-22 15:06:21.980419: I tensorflow/core/profiler/lib/profiler_session.cc:164] Profiler session tear down.


  156/30000 [..............................] - ETA: 38s - loss: 0.1170 - binary_accuracy: 0.9635

2022-04-22 15:06:22.460579: I tensorflow/core/profiler/lib/profiler_session.cc:131] Profiler session initializing.
2022-04-22 15:06:22.460591: I tensorflow/core/profiler/lib/profiler_session.cc:146] Profiler session started.
2022-04-22 15:06:22.463757: I tensorflow/core/profiler/lib/profiler_session.cc:66] Profiler session collecting data.
2022-04-22 15:06:22.466241: I tensorflow/core/profiler/lib/profiler_session.cc:164] Profiler session tear down.
2022-04-22 15:06:22.468900: I tensorflow/core/profiler/rpc/client/save_profile.cc:136] Creating directory: ./pipeline/Trainer/model_run/7/train/plugins/profile/2022_04_22_15_06_22

2022-04-22 15:06:22.469880: I tensorflow/core/profiler/rpc/client/save_profile.cc:142] Dumped gzipped tool data for trace.json.gz to ./pipeline/Trainer/model_run/7/train/plugins/profile/2022_04_22_15_06_22/hongjin-uui-MacBookPro.local.trace.json.gz
2022-04-22 15:06:22.472699: I tensorflow/core/profiler/rpc/client/save_profile.cc:136] Creating directory: ./pipelin

30000/30000 [==============================] - 51s 2ms/step - loss: 0.0024 - binary_accuracy: 0.9995 - val_loss: 0.2473 - val_binary_accuracy: 0.9715
INFO:tensorflow:tensorflow_text is not available.
INFO:tensorflow:tensorflow_decision_forests is not available.
INFO:tensorflow:struct2tensor is not available.
INFO:tensorflow:Assets written to: ./pipeline/Trainer/model/7/Format-Serving/assets


ExecutionResult(
    component_id: Trainer
    execution_id: 7
    outputs:
        model: Channel(
            type_name: Model
            artifacts: [Artifact(artifact: id: 14
        type_id: 27
        uri: "./pipeline/Trainer/model/7"
        custom_properties {
          key: "name"
          value {
            string_value: "model"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "Trainer"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "1.3.0"
          }
        }
        state: LIVE
        , artifact_type: id: 27
        name: "Model"
        )]
            additional_properties: {}
            additional_custom_properties: {}
        )
        model_run: Channel(
            type_name: ModelRun
            artifacts: [Artifact(artifact: id: 15
        type_id: 28
        uri: "./pipeline/Trainer/model_run/7"
        custom_properties {
          key: "name"
          value {
            string_value: "model_run"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "Trainer"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "1.3.0"
          }
        }
        state: LIVE
        , artifact_type: id: 28
        name: "ModelRun"
        )]
            additional_properties: {}
            additional_custom_properties: {}
        ))

In [49]:
# Get `model` output of the component
model_artifact_dir = trainer.outputs['model'].get()[0].uri

# List top-level directory
pp.pprint(os.listdir(model_artifact_dir))

# Get model directory
model_dir = os.path.join(model_artifact_dir, 'Format-Serving')

# List subdirectories
pp.pprint(os.listdir(model_dir))

['Format-Serving']
['keras_metadata.pb', 'variables', 'saved_model.pb', 'assets']


In [50]:
# Get `model_run` output URI
model_run_artifact_dir = trainer.outputs['model_run'].get()[0].uri

# Load results to Tensorboard
%load_ext tensorboard
%tensorboard --logdir {model_run_artifact_dir}

Reusing TensorBoard on port 6006 (pid 60842), started 20:20:57 ago. (Use '!kill 60842' to kill it.)

---

#### 일반적인 상황 파이프라인 (Tuner 없이 그대로)

In [51]:
# from tensorflow import keras
# from typing import NamedTuple, Dict, Text, Any, List
# from tfx.components.trainer.fn_args_utils import FnArgs, DataAccessor
# import tensorflow as tf
# import tensorflow_transform as tft
# import advert_constants as constants


# _DENSE_FLOAT_FEATURE_KEYS = constants.DENSE_FLOAT_FEATURE_KEYS
# _VOCAB_FEATURE_KEYS = constants.VOCAB_FEATURE_KEYS
# _VOCAB_SIZE = constants.VOCAB_SIZE
# _OOV_SIZE = constants.OOV_SIZE
# _LABEL_KEY = constants.LABEL_KEY
# _transformed_name = constants.transformed_name


# def _transformed_names(keys):
#     return [_transformed_name(key) for key in keys]


# def _gzip_reader_fn(filenames):
#     """Load compressed dataset

#     Args:
#       filenames - filenames of TFRecords to load

#     Returns:
#       TFRecordDataset loaded from the filenames
#     """

#     # Load the dataset. Specify the compression type since it is saved as `.gz`
#     return tf.data.TFRecordDataset(filenames, compression_type='GZIP')


# def _input_fn(file_pattern,
#               tf_transform_output,
#               num_epochs=1,
#               batch_size=32) -> tf.data.Dataset:
#     """Create batches of features and labels from TF Records

#     Args:
#       file_pattern - List of files or patterns of file paths containing Example records.
#       tf_transform_output - transform output graph
#       num_epochs - Integer specifying the number of times to read through the dataset.
#               If None, cycles through the dataset forever.
#       batch_size - An int representing the number of records to combine in a single batch.

#     Returns:
#       A dataset of dict elements, (or a tuple of dict elements and label).
#       Each dict maps feature keys to Tensor or SparseTensor objects.
#     """
#     transformed_feature_spec = (
#         tf_transform_output.transformed_feature_spec().copy())

#     dataset = tf.data.experimental.make_batched_features_dataset(
#         file_pattern=file_pattern,
#         batch_size=batch_size,
#         features=transformed_feature_spec,
#         reader=_gzip_reader_fn,
#         num_epochs=num_epochs,
#         label_key=_transformed_name(_LABEL_KEY))

#     return dataset


# def _get_serve_tf_examples_fn(model, tf_transform_output):
#     """Returns a function that parses a serialized tf.Example and applies TFT."""

#     # Get transformation graph
#     model.tft_layer = tf_transform_output.transform_features_layer()

#     @tf.function
#     def serve_tf_examples_fn(serialized_tf_examples):
#         """Returns the output to be used in the serving signature."""
#         # Get pre-transform feature spec
#         feature_spec = tf_transform_output.raw_feature_spec()

#         # Pop label since serving inputs do not include the label
#         feature_spec.pop(_LABEL_KEY)

#         # Parse raw examples into a dictionary of tensors matching the feature spec
#         parsed_features = tf.io.parse_example(serialized_tf_examples, feature_spec)

#         # Transform the raw examples using the transform graph
#         transformed_features = model.tft_layer(parsed_features)

#         # Get predictions using the transformed features
#         return model(transformed_features)

#     return serve_tf_examples_fn


# def _model_builder(hp):
#     """
#     Builds the model and sets up the hyperparameters to tune.

#     Args:
#       hp - Keras tuner object

#     Returns:
#       model with hyperparameters to tune
#     """

#     num_dnn_layers = 3  # 일단 복잡하니까 작은 수로 지정하고 optimize도 제외
#     hp_first_units = hp.Int('units', min_value=128, max_value=256, step=32)
#     hp_decay_factor = hp.Choice('decay', values=[0.7, 0.5, 0.3])

#     hidden_units = [
#          max(2, int(hp_first_units * hp_decay_factor**i))
#          for i in range(num_dnn_layers)
#     ]

#     model = _wide_and_deep_classifier(hidden_units)

#     return model


# def _wide_and_deep_classifier(hp):
#     input_numeric = [
#         tf.keras.layers.Input(name=colname, shape=(1,), dtype=tf.float32)
#         for colname in _transformed_names(_DENSE_FLOAT_FEATURE_KEYS)
#     ]
#     input_categorical = [
#         tf.keras.layers.Input(name=colname, shape=(_VOCAB_SIZE + _OOV_SIZE,), dtype=tf.float32)
#         for colname in _transformed_names(_VOCAB_FEATURE_KEYS)
#     ]

#     # TODO: Tuner hyperparameters로 받아오게 변경
#     # deep 부분 정의
#     deep = tf.keras.layers.concatenate(input_numeric)
#     for numnodes in hidden_units:
#         deep = tf.keras.layers.Dense(numnodes, name='deep_'+str(numnodes))(deep)
#     # wide 부분 정의
#     wide = tf.keras.layers.concatenate(input_categorical)
#     # concat
#     combined = keras.layers.concatenate([wide, deep], name='combined')
#     # output 만들기
#     output = tf.keras.layers.Dense(1, activation='sigmoid', name='output')(combined)
#     # input layers와 output을 넣어 model 만들기
#     # 이런 식으로 만들면 input layer에서 분리되서 wide, deep 나뉘고 다시 output으로 합쳐지게 깔끔하게 나옴
#     input_layers = input_numeric + input_categorical
#     model = tf.keras.Model(input_layers, output)

#     # TODO: learning rate Tuner hyperparameters로 변경
#     # Setup model for training
#     model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.01),
#                   loss="binary_crossentropy",
#                   metrics=[tf.keras.metrics.BinaryAccuracy()])

#     # Print the model summary
#     model.summary()

#     return model


# def run_fn(fn_args: FnArgs) -> None:
#     """Defines and trains the model.
#     Args:
#       fn_args: Holds args as name/value pairs. Refer here for the complete attributes:
#       https://www.tensorflow.org/tfx/api_docs/python/tfx/components/trainer/fn_args_utils/FnArgs#attributes
#     """

#     # Callback for TensorBoard
#     tensorboard_callback = tf.keras.callbacks.TensorBoard(
#         log_dir=fn_args.model_run_dir, update_freq='batch')

#     # Load transform output
#     tf_transform_output = tft.TFTransformOutput(fn_args.transform_graph_path)

#     train_set = _input_fn(fn_args.train_files[0], tf_transform_output, num_epochs=5)  # 여기서 받아올 epochs 설정되어 데이터 순회됨
#     val_set = _input_fn(fn_args.eval_files[0], tf_transform_output, num_epochs=5)

#     # TODO: 여기서 tuner 이용하게 바꾸기
#     # Load best hyperparameters
#     # hp = fn_args.hyperparameters.get('values')
#     hp = {'hidden_units': [10, 20],
#           'learning_rate': 1e-3}

#     # Build the model
#     model = _model_builder(hp)

#     # Train the model
#     model.fit(
#         train_set,
#         steps_per_epoch=fn_args.train_steps,  # 이렇게 해야 num_steps을 받아옴
#         validation_data=val_set,
#         validation_steps=fn_args.eval_steps,  # 이렇게 해야 num_steps을 받아옴
#         callbacks=[tensorboard_callback])

#     # Define default serving signature
#     # TODO: 조금 더 제대로 이해해보기
#     # SavedModel에 있는 하나의 특성으로 이를 바탕으로 raw data부터 전처리까지 한 후 추론이 가능하도록 만들 수 있음
#     # _get_serve_tf_examples_fn 내에 정의된 serve_tf_example_fn(tf.function)으로 get_concrete_function을 통해 examples 데이터를 전달해 추론 가능하도록 함
#     signatures = {
#       'serving_default':
#           _get_serve_tf_examples_fn(model,
#                                     tf_transform_output).get_concrete_function(
#                                         tf.TensorSpec(
#                                             shape=[None],
#                                             dtype=tf.string,
#                                             name='examples')),
#     }

#     # Save the model
#     model.save(fn_args.serving_model_dir, save_format='tf', signatures=signatures)

In [52]:
# advert_trainer = 'model/advert_trainer.py'
# trainer = Trainer(
#    module_file=advert_trainer,
#    examples=transform.outputs['transformed_examples'],
#    transform_graph=transform.outputs['transform_graph'],
#    schema=schema_gen.outputs['schema'],
#    train_args=trainer_pb2.TrainArgs(num_steps=500),  # batch 단위로 num_steps만큼 학습
#    eval_args=trainer_pb2.EvalArgs(num_steps=250))

In [53]:
# context.run(trainer)

In [54]:
# # Get `model` output of the component
# model_artifact_dir = trainer.outputs['model'].get()[0].uri

# # List top-level directory
# pp.pprint(os.listdir(model_artifact_dir))

# # Get model directory
# model_dir = os.path.join(model_artifact_dir, 'Format-Serving')

# # List subdirectories
# pp.pprint(os.listdir(model_dir))

In [55]:
# # Get `model_run` output URI
# model_run_artifact_dir = trainer.outputs['model_run'].get()[0].uri

# # Load results to Tensorboard
# %load_ext tensorboard
# %tensorboard --logdir {model_run_artifact_dir}

---

## Evaluator

In [77]:
eval_config = text_format.Parse("""
  ## Model information
  model_specs {
    # This assumes a serving model with signature 'serving_default'.
    signature_name: "serving_default",
    label_key: "ClickedOnAd"
  }

  ## Post training metric information
  metrics_specs {
    metrics { class_name: "ExampleCount" }
    metrics {
      class_name: "BinaryAccuracy"
      threshold {
        # Ensure that metric is always > 0.5
        value_threshold {
          lower_bound { value: 0.5 }
        }
        # Ensure that metric does not drop by more than a small epsilon
        # e.g. (candidate - baseline) > -1e-10 or candidate > baseline - 1e-10
        change_threshold {
          direction: HIGHER_IS_BETTER
          absolute { value: -1e-10 }
        }
      }
    }
    metrics { class_name: "BinaryCrossentropy" }
    metrics { class_name: "AUC" }
    metrics { class_name: "AUCPrecisionRecall" }
    metrics { class_name: "Precision" }
    metrics { class_name: "Recall" }
    # ... add additional metrics and plots ...
  }

  ## Slicing information
  # slicing_specs {}  # overall slice
  # slicing_specs {
  #  feature_keys: ["Male"]
  # }
  # slicing_specs {
  #  feature_values: {
  #    key: "Country"
  #    value: "Senegal"}
  # }
  # slicing_specs {
  #   feature_keys: ["Male", "Age"]
  # }
  
""", tfma.EvalConfig())

In [78]:
# 마지막 blessed model 찾기 위한 코드
# Setup the Resolver node to find the latest blessed model
model_resolver = Resolver(
      strategy_class=tfx.v1.dsl.experimental.LatestBlessedModelStrategy,
      model=tfx.v1.dsl.Channel(type=tfx.types.standard_artifacts.Model),
      model_blessing=tfx.v1.dsl.Channel(
          type=tfx.types.standard_artifacts.ModelBlessing)).with_id(
              'latest_blessed_model_resolver')

# Run the Resolver node
context.run(model_resolver)

ExecutionResult(
    component_id: latest_blessed_model_resolver
    execution_id: 13
    outputs:
        model: Channel(
            type_name: Model
            artifacts: []
            additional_properties: {}
            additional_custom_properties: {}
        )
        model_blessing: Channel(
            type_name: ModelBlessing
            artifacts: []
            additional_properties: {}
            additional_custom_properties: {}
        ))

In [79]:
# output 중 evaluation에 metrics, slicing 등 설정에 따른 평가 결과, model_blessing에 모델이 평가기준 통과했는지 BLESSED, NOT_BLESSED로 출력
evaluator = Evaluator(
   examples=example_gen.outputs['examples'],
   model=trainer.outputs['model'],
   baseline_model=model_resolver.outputs['model'],
   eval_config=eval_config)
context.run(evaluator, enable_cache=False)


Two checkpoint references resolved to different objects (<keras.saving.saved_model.load.TensorFlowTransform>TransformFeaturesLayer object at 0x1a6856340> and <keras.engine.input_layer.InputLayer object at 0x1a5b3bc40>).

Two checkpoint references resolved to different objects (<keras.saving.saved_model.load.TensorFlowTransform>TransformFeaturesLayer object at 0x1ac16a100> and <keras.engine.input_layer.InputLayer object at 0x1a63f81f0>).



Two checkpoint references resolved to different objects (<keras.saving.saved_model.load.TensorFlowTransform>TransformFeaturesLayer object at 0x1a6d7b880> and <keras.engine.input_layer.InputLayer object at 0x1a6d96a30>).

Two checkpoint references resolved to different objects (<keras.saving.saved_model.load.TensorFlowTransform>TransformFeaturesLayer object at 0x1a6af6e80> and <keras.engine.input_layer.InputLayer object at 0x1a72850a0>).

Two checkpoint references resolved to different objects (<keras.saving.saved_model.load.TensorFlowTransform>TransformFeaturesLayer object at 0x1acb74c70> and <keras.engine.input_layer.InputLayer object at 0x1acb6e250>).

Two checkpoint references resolved to different objects (<keras.saving.saved_model.load.TensorFlowTransform>TransformFeaturesLayer object at 0x1aca6deb0> and <keras.engine.input_layer.InputLayer object at 0x1a6c5f4c0>).

Two checkpoint references resolved to different objects (<keras.saving.saved_model.load.TensorFlowTransform>Transfo

ExecutionResult(
    component_id: Evaluator
    execution_id: 14
    outputs:
        evaluation: Channel(
            type_name: ModelEvaluation
            artifacts: [Artifact(artifact: id: 21
        type_id: 31
        uri: "./pipeline/Evaluator/evaluation/14"
        custom_properties {
          key: "name"
          value {
            string_value: "evaluation"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "Evaluator"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "1.3.0"
          }
        }
        state: LIVE
        , artifact_type: id: 31
        name: "ModelEvaluation"
        )]
            additional_properties: {}
            additional_custom_properties: {}
        )
        blessing: Channel(
            type_name: ModelBlessing
            artifacts: [Artifact(artifact: id: 22
        type_id: 32
        uri: "./pipeline/Evaluator/blessing/14"
        custom_properties {
          key: "blessed"
          value {
            int_value: 1
          }
        }
        custom_properties {
          key: "current_model"
          value {
            string_value: "./pipeline/Trainer/model/7"
          }
        }
        custom_properties {
          key: "current_model_id"
          value {
            int_value: 14
          }
        }
        custom_properties {
          key: "name"
          value {
            string_value: "blessing"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "Evaluator"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "1.3.0"
          }
        }
        state: LIVE
        , artifact_type: id: 32
        name: "ModelBlessing"
        )]
            additional_properties: {}
            additional_custom_properties: {}
        ))

In [80]:
evaluator.outputs.keys()

dict_keys(['evaluation', 'blessing'])

In [81]:
# Bless된 모델인지 확인 (이전 모델보다 성능이 threshold를 충족할만큼 나아졌는지)
# Get `Evaluator` blessing output URI
blessing_uri = evaluator.outputs['blessing'].get()[0].uri
print(blessing_uri)
# List files under URI
os.listdir(blessing_uri)

./pipeline/Evaluator/blessing/14


['BLESSED']

In [84]:
import tensorflow_model_analysis as tfma

# TODO: slicing도 evaluation 기준을 모두 만족해야 blessed가 되나? 그렇다면 production에서는 꼭 필요한 슬라이스만 설정하고 자세하게 보고 싶을 때는 evaluator를 따로 돌려야 할 듯

# TFMA 시각화 사용방법
# Examples Threshold: slice 중에 너무 작은 slice 제외 (Slices Overview, Metrics Histogram 공통적으로 적용됨)

# Slices Overview vs Metrics Histogram
# Slices Overview: 정해진 slice별로 auc, accuracy 등 파악 가능
# Show를 통해 원하는 metrics 선택, Sort by를 통해 원하는 metrics별로 sort (show랑 sort by는 별도로 돌아감)

# Metrics Histogram: Slice를 정해진 buckets(default=10)으로 나누어 원하는 metrics에 대해 bucket별 slice counts나 example counts 시각화
# Number of Buckets: 나눌 Histogram bucket 수
# Select Metric: 원하는 metrics 선택
# Histogram Type: Slice Counts(히스토그램별 슬라이스 카운트), Example Counts(히스토그램별 속한 데이터 갯수), Both (두 개 다 보여줌)

# Get the TFMA output result path and load the result.
PATH_TO_RESULT = evaluator.outputs['evaluation'].get()[0].uri
tfma_result = tfma.load_eval_result(PATH_TO_RESULT)

# Show data sliced along feature column trip_start_hour.
# slicing_column을 통해 만들어놓은 컬럼을 직접 지정할 수도 있고 slicing_spec을 써서 config에 있는 값 불러올 수 있음
tfma.view.render_slicing_metrics(tfma_result)
# tfma.view.render_slicing_metrics(tfma_result, slicing_column='City')
# tfma.view.render_slicing_metrics(tfma_result, slicing_column='Male')
# tfma.view.render_slicing_metrics(tfma_result, 
#                                 slicing_spec=tfma.SlicingSpec(
#                                     feature_keys=['Male', 'Age']))  # 만들어 놓았다면 male * age 의 slice를 보여줌
# tfma.view.render_slicing_metrics(tfma_result, 
#                                  slicing_spec=tfma.SlicingSpec(
#                                      feature_keys=['Country'], feature_values={'Country': 'Korea'}))
# tfma.view.render_slicing_metrics(tfma_result, 
#                                  slicing_spec=tfma.SlicingSpec(
#                                      feature_values={'Country': 'Senegal'}))

# error 예시
# tfma.view.render_slicing_metrics(tfma_result, slicing_column='AdTopicLine') # AdTopicLine에 대해 만들지 않아서 나오지 않음

SlicingMetricsViewer(config={'weightedExamplesColumn': 'example_count'}, data=[{'slice': 'Overall', 'metrics':…

In [85]:
# BLESSED 되었는지 확인
validation_result = tfma.load_validation_result(PATH_TO_RESULT)
print(validation_result.validation_ok)

True


## Pusher

In [86]:
evaluator.outputs['blessing']

Channel(
    type_name: ModelBlessing
    artifacts: [Artifact(artifact: id: 22
type_id: 32
uri: "./pipeline/Evaluator/blessing/14"
custom_properties {
  key: "blessed"
  value {
    int_value: 1
  }
}
custom_properties {
  key: "current_model"
  value {
    string_value: "./pipeline/Trainer/model/7"
  }
}
custom_properties {
  key: "current_model_id"
  value {
    int_value: 14
  }
}
custom_properties {
  key: "name"
  value {
    string_value: "blessing"
  }
}
custom_properties {
  key: "producer_component"
  value {
    string_value: "Evaluator"
  }
}
custom_properties {
  key: "state"
  value {
    string_value: "published"
  }
}
custom_properties {
  key: "tfx_version"
  value {
    string_value: "1.3.0"
  }
}
state: LIVE
, artifact_type: id: 32
name: "ModelBlessing"
)]
    additional_properties: {}
    additional_custom_properties: {}
)

In [87]:
serving_model_dir = 'serving_model/advert-pred'
pusher = Pusher(
    model=trainer.outputs['model'],
    model_blessing=evaluator.outputs['blessing'],
    push_destination=pusher_pb2.PushDestination(
    filesystem=pusher_pb2.PushDestination.Filesystem(
    base_directory=serving_model_dir)))

In [88]:
context.run(pusher)

ExecutionResult(
    component_id: Pusher
    execution_id: 15
    outputs:
        pushed_model: Channel(
            type_name: PushedModel
            artifacts: [Artifact(artifact: id: 23
        type_id: 34
        uri: "./pipeline/Pusher/pushed_model/15"
        custom_properties {
          key: "name"
          value {
            string_value: "pushed_model"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "Pusher"
          }
        }
        custom_properties {
          key: "pushed"
          value {
            int_value: 1
          }
        }
        custom_properties {
          key: "pushed_destination"
          value {
            string_value: "serving_model/advert-pred/1650611145"
          }
        }
        custom_properties {
          key: "pushed_version"
          value {
            string_value: "1650611145"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "1.3.0"
          }
        }
        state: LIVE
        , artifact_type: id: 34
        name: "PushedModel"
        )]
            additional_properties: {}
            additional_custom_properties: {}
        ))